In [15]:
from src.sqlstore.db import connect_to_db

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Webscraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

import time

In [16]:
engine = connect_to_db("../src/database.ini")

In [18]:
match = "SELECT * FROM match"

stats = "SELECT * FROM match_participant_stats"

challenges = "SELECT * FROM match_participant_challenges"

In [19]:
df_match = pd.read_sql_query(match, engine)

df_stats = pd.read_sql_query(stats, engine)

df_challenges = pd.read_sql_query(challenges, engine)

In [20]:
df_combined = pd.merge(df_match, df_stats, on='gameId', how='inner')

In [21]:
df = final_combined = df_combined.merge(df_challenges, on=['puuid', 'gameId'], how='inner')

In [22]:
# Drop unnecessary columns
df = df.drop(columns=['matchId', 'platformId_x', 'gameId', 'seasonId', 'patch', 'queueId', 'gameVersion', 'mapId', 'mapId', 'gameCreation', 'gameCreation', 'timeCreated_x', 'lastUpdate_x', 'puuid', 'timeCreated', 'lastUpdate', 'platformId_y'])

In [23]:
df.head()

,gameDuration,allInPings,assistMePings,assists,baitPings,baronKills,bountyLevel,champExperience,champLevel,championId,...,turretTakedowns_y,turretsTakenWithRiftHerald,twentyMinionsIn3SecondsCount,twoWardsOneSweeperCount,unseenRecalls,visionScoreAdvantageLaneOpponent,visionScorePerMinute,wardTakedowns,wardTakedownsBefore20M,wardsGuarded
0,1686,0,1,9,0,0,1,17221,17,897,...,3,5,9,0,0,0.466623,0.820007,2,2,0
1,1686,2,6,8,3,2,3,15768,16,76,...,4,5,1,0,0,0.466252,1.342831,9,6,0
2,1686,1,3,4,0,0,0,13470,15,246,...,1,5,7,0,0,-0.049789,0.502217,2,0,0
3,1686,0,0,11,1,0,3,16162,16,119,...,7,5,11,0,0,1.008010,1.125961,5,3,0
4,1686,1,8,18,1,0,0,10930,13,12,...,5,5,0,1,0,-0.118339,2.314497,9,8,0


In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

url = "https://u.gg/lol/tier-list"

options = Options()
options.add_argument("--headless")
options.binary_location = "C:\\Users\\nicol\\Downloads\\chrome-win64\\chrome-win64\\chrome.exe"  # Path to your Chrome executable

service = Service("C:\\Users\\nicol\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.rt-tr-group")))

data = []  # List to store each row of data

while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    
    # Scroll back to the top of the page
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(1)
    
    # Check if all rows have been loaded
    rows = driver.find_elements(By.CSS_SELECTOR, "div.rt-tr-group")
    if not rows or len(data) == len(rows):  # Adjusted condition to check data length instead of champions length
        break
    
    # Otherwise, continue to extract the data
    for i in range(len(data), len(rows)):
        row = rows[i]
        try:
            rank = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(1)").text.strip()
            champion = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(3)").get_attribute("textContent")
            tier = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(4)").text.strip()
            win_rate = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(5)").text.strip()
            pick_rate = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(7)").text.strip()
            ban_rate = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(6)").text.strip()
            matches = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(8)").text.strip()
    
            # Append the extracted data to the data list
            data.append([rank, champion, tier, win_rate, pick_rate,ban_rate, matches])
        
        except Exception as e:
            print(f"Error in row {i}: {e}")


# Convert the data list into a pandas DataFrame
columns = ['Rank', 'Champion Name', 'Tier', 'Win rate', 'Pick Rate', 'Ban Rate', 'Matches']
df_scraped = pd.DataFrame(data, columns=columns)

driver.quit()

In [25]:
df_scraped.head()

,Rank,Champion Name,Tier,Win rate,Pick Rate,Ban Rate,Matches
0,1,Swain,S,53.87%,,1.7%,"19,216"
1,2,Quinn,S,53.57%,,1.8%,"36,299"
2,3,Karthus,S,53.56%,,5.5%,"21,460"
3,4,Tryndamere,S,53.29%,,13.1%,"19,945"
4,5,Rek'Sai,S+,53.23%,,13.6%,"71,269"


In [26]:
df_scraped.to_csv(r"D:\Datensätze\Leaguify\test_scraped.csv")

In [27]:
# Convert 'Matches' column to integers for calculation, as it's currently in string format with commas
df_scraped['Matches'] = df_scraped['Matches'].str.replace(',', '').astype(int)

# Calculate the total number of matches

total_matches = df_scraped['Matches'].sum()

# Calculate the playrate (Pick Rate) for each champion
df_scraped['Pick Rate'] = (df_scraped['Matches'] / total_matches) * 100

# Convert the calculated playrate to percentage format for better readability
df_scraped['Pick Rate'] = df_scraped['Pick Rate'].apply(lambda x: f"{x:.2f}%")

# Remove the '%' symbol from 'Win rate' and 'Ban Rate' columns and convert them to float
df_scraped['Win rate'] = df_scraped['Win rate'].str.rstrip('%').astype('float')
df_scraped['Ban Rate'] = df_scraped['Ban Rate'].str.rstrip('%').astype('float')

# Remove the '%' symbol from the newly calculated 'Pick Rate' and convert it to float
df_scraped['Pick Rate'] = df_scraped['Pick Rate'].str.rstrip('%').astype('float')

# Convert the 'Pick Rate' from fraction to the format like 'Win rate' (e.g., 0.55 to 55.00)
df_scraped['Pick Rate'] = df_scraped['Pick Rate'] * 100

# Show the updated DataFrame
df_scraped.head()

,Rank,Champion Name,Tier,Win rate,Pick Rate,Ban Rate,Matches
0,1,Swain,S,53.87,12.0,1.7,19216
1,2,Quinn,S,53.57,23.0,1.8,36299
2,3,Karthus,S,53.56,13.0,5.5,21460
3,4,Tryndamere,S,53.29,12.0,13.1,19945
4,5,Rek'Sai,S+,53.23,44.0,13.6,71269


In [30]:
# Combine the two DataFrames based on the "Champion Name" column
df_combined = pd.merge(df_scraped, df, left_on='Champion Name', right_on='championName', how='inner')

# Show the first few rows of the combined DataFrame
df_combined.head()

,Rank,Champion Name,Tier,Win rate,Pick Rate,Ban Rate,Matches,gameDuration,allInPings,assistMePings,...,turretTakedowns_y,turretsTakenWithRiftHerald,twentyMinionsIn3SecondsCount,twoWardsOneSweeperCount,unseenRecalls,visionScoreAdvantageLaneOpponent,visionScorePerMinute,wardTakedowns,wardTakedownsBefore20M,wardsGuarded
0,1,Swain,S,53.87,12.0,1.7,19216,1711,0,9,...,7,0,4,0,0,0.731238,1.166228,4,2,1
1,1,Swain,S,53.87,12.0,1.7,19216,1987,0,7,...,1,0,9,0,0,-0.407494,0.594141,1,1,0
2,1,Swain,S,53.87,12.0,1.7,19216,1495,0,2,...,1,0,8,0,0,-0.782050,0.388326,1,0,0
3,1,Swain,S,53.87,12.0,1.7,19216,1528,1,3,...,1,5,6,0,0,-0.055465,0.637202,4,3,0
4,1,Swain,S,53.87,12.0,1.7,19216,1155,0,0,...,0,0,1,0,0,-0.218048,1.941728,1,1,0


In [32]:
df_combined.to_csv(r"D:\Datensätze\Leaguify\combined.csv")